In [29]:
import tensorflow as tf

# データの読み込み
# https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image_dataset_from_directory
# 画像サイズが元々75x75なので、2倍くらい近いの160x160にリサイズした。
train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    "../dog_cat_photos/train",
    image_size=(160, 160),
    label_mode="binary",
#     batch_size=32,
    shuffle=True
)

test_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    "../dog_cat_photos/test",
    image_size=(160, 160),
    label_mode="binary",
#     batch_size=32,
    shuffle=False
)


Found 300 files belonging to 2 classes.
Found 100 files belonging to 2 classes.


In [30]:
# 画像の水増しをする関数の定義（コピペ）
def flip_left_right(image, label):   # 左右反転
    image = tf.image.flip_left_right(image)
    return image, label

def flip_up_down(image, label):      # 上下反転
    image = tf.image.flip_up_down(image)
    return image, label

def rot90(image, label):             # 反時計回りに90度回転
    image = tf.image.rot90(image)
    return image, label

def rot180(image, label):            # 反時計回りに180度回転
    image = tf.image.rot90(image, k=2)
    return image, label

def rot270(image, label):            # 反時計回りに270度回転
    image = tf.image.rot90(image, k=3)
    return image, label

In [31]:
class_names = train_dataset.class_names
class_names # ['cat', 'dog']

# データの水増しの処理
train_dataset_lr     = train_dataset.map(flip_left_right)
train_dataset_ud     = train_dataset.map(flip_up_down)
train_dataset_rot90  = train_dataset.map(rot90)
train_dataset_rot180 = train_dataset.map(rot180)
train_dataset_rot270 = train_dataset.map(rot270)

# 水増しデータの結合
train_dataset = train_dataset.concatenate(train_dataset_lr)
train_dataset = train_dataset.concatenate(train_dataset_ud)
train_dataset = train_dataset.concatenate(train_dataset_rot90)
train_dataset = train_dataset.concatenate(train_dataset_rot180)
train_dataset = train_dataset.concatenate(train_dataset_rot270)

# またシャフルします
train_dataset = train_dataset.shuffle(32)

# 画像の表示チェック（回転後の画像もでます）
# import matplotlib.pyplot as plt
# 
# plt.figure(figsize=(10, 10))
# 
# for images, labels in train_dataset.take(1):
#     for i in range(9):
#         ax = plt.subplot(3, 3, i + 1)
#         plt.imshow(images[i].numpy().astype("uint8"))
#         plt.title(class_names[labels[i].numpy().astype("uint8")[0]])


In [ ]:
# リサイズ処理はimage_dataset_from_directoryで行われているので、ここではデータの水増しのみを行う

# MobileNetV2 のモデルを利用
input_layer = tf.keras.Input(shape=(160, 160, 3))   # 入力層
l_layer = tf.keras.applications.mobilenet_v2.preprocess_input(input_layer)   # 前処理（正規化）をする層

# MobileNetV2の重みファイルのパス
# ここでは、事前にダウンロードしておいた重みファイルを指定する。
# 指定する原因は、"imagenet"でダウンロードするとSSLエラーが出られます。
weights_path = './mobilenet_v2_weights_tf_dim_ordering_tf_kernels_1.0_224_no_top.h5'
base_model = tf.keras.applications.mobilenet_v2.MobileNetV2(
    input_shape=(160, 160, 3),
    input_tensor=l_layer,
    include_top=False,
    weights=weights_path,
    pooling='avg'
)

base_model.trainable = False

# Dense層を追加する
output_layer = tf.keras.layers.Dense(1, activation='sigmoid')

# base_modelに先ほどのDense層を追加したモデルを作成する
model = tf.keras.Sequential([
    base_model,
    output_layer
])

# modelをcompileする
model.compile(optimizer="adam",
              loss='binary_crossentropy',
              metrics=["accuracy"])

# modelに学習させる
model.fit(train_dataset, epochs=20)

# テストデータで分類を実行する
pred_data = model.predict(test_dataset)
pred_data

Epoch 1/20
60/60 ━━━━━━━━━━━━━━━━━━━━ 5s 66ms/step - accuracy: 0.7147 - loss: 0.5282
Epoch 2/20
60/60 ━━━━━━━━━━━━━━━━━━━━ 4s 66ms/step - accuracy: 0.9220 - loss: 0.1804
Epoch 3/20
60/60 ━━━━━━━━━━━━━━━━━━━━ 4s 65ms/step - accuracy: 0.9612 - loss: 0.1176
Epoch 4/20
60/60 ━━━━━━━━━━━━━━━━━━━━ 4s 65ms/step - accuracy: 0.9779 - loss: 0.0862
Epoch 5/20
60/60 ━━━━━━━━━━━━━━━━━━━━ 4s 68ms/step - accuracy: 0.9759 - loss: 0.0824
Epoch 6/20
60/60 ━━━━━━━━━━━━━━━━━━━━ 4s 70ms/step - accuracy: 0.9819 - loss: 0.0684
Epoch 7/20
60/60 ━━━━━━━━━━━━━━━━━━━━ 4s 69ms/step - accuracy: 0.9880 - loss: 0.0577
Epoch 8/20
60/60 ━━━━━━━━━━━━━━━━━━━━ 4s 65ms/step - accuracy: 0.9962 - loss: 0.0486
Epoch 9/20
60/60 ━━━━━━━━━━━━━━━━━━━━ 4s 67ms/step - accuracy: 0.9891 - loss: 0.0492
Epoch 10/20
60/60 ━━━━━━━━━━━━━━━━━━━━ 4s 67ms/step - accuracy: 0.9955 - loss: 0.0363
Epoch 11/20
60/60 ━━━━━━━━━━━━━━━━━━━━ 4s 68ms/step - accuracy: 0.9981 - loss: 0.0315
Epoch 12/20
60/60 ━━━━━━━━━━━━━━━━━━━━ 4s 68ms/step - accuracy:

array([[3.56452860e-04],
       [9.94075090e-05],
       [8.42718524e-04],
       [6.68382621e-04],
       [1.18076845e-04],
       [3.70117807e-04],
       [1.58082439e-05],
       [4.61969496e-04],
       [5.36618290e-06],
       [2.14216736e-04],
       [3.33827265e-05],
       [5.91541082e-03],
       [5.41372174e-05],
       [7.27386141e-05],
       [3.79653866e-05],
       [5.01459144e-05],
       [2.73665559e-04],
       [1.82001331e-05],
       [7.07216968e-05],
       [1.95712084e-03],
       [6.74995135e-06],
       [4.31366591e-03],
       [3.78720870e-05],
       [1.26945204e-04],
       [4.36035275e-01],
       [1.57812145e-04],
       [3.11875367e-03],
       [3.42462845e-05],
       [9.74697003e-04],
       [6.08316695e-05],
       [1.54428827e-02],
       [8.57832849e-01],
       [3.26351392e-06],
       [1.36084752e-02],
       [3.51802795e-04],
       [9.86474276e-01],
       [1.04535501e-02],
       [2.62735819e-04],
       [5.08262496e-03],
       [4.08142398e-04],


In [33]:
# 正答率のチェック
model.evaluate(test_dataset)

4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 54ms/step - accuracy: 0.9661 - loss: 0.0971


[0.08927605301141739, 0.9700000286102295]